# Import external and internal modules

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
from mpl_toolkits.basemap import Basemap
import math
import zipfile
import os

In [1]:
from modules.utils import *
# importlib.reload(modules.utils)
from modules.utils import *

# Read grid data

In [3]:
lat_csv = 'data/latitude_EASE.csv'
lon_csv = 'data/longitude_EASE.csv'
grid = generate_ease_grid(lat_csv, lon_csv)
print(grid.shape)

(361, 361, 2)


# Draw the EASE grid on map

In [ ]:
# earth view
fig = plt.figure(figsize=(8, 8), edgecolor='w')
draw_map([], [], [], [], grid, fig, projection='ortho')

In [ ]:
# zoomed view
fig = plt.figure(figsize=(8, 8), edgecolor='w')
draw_map([], [], [], [], grid, fig, projection='stere', grid_res=4, width=5000000, height=5000000)

# Read the sea ice dataset and clean it up

In [8]:
data = read_dataset('data/DRIFT_DATA_TRAIN.csv')
#print(data.describe())
'''
data cleaning up
'''
# Remove any rows that have buoy velocity/mag =0 
data = data.drop(data[data["u_buoy"]*data["v_buoy"] == 0].index)
data = data.reset_index()

# average the ice thickness from two measurements
temp = [None]*data.shape[0]
# iterate through all the rows and check if thickness is 0
for index, row in data.iterrows():
    if row["h_piomas"] == 0 :  
        if row["h_cs2smos"] > 0: 
            temp[index] = row["h_cs2smos"]
        elif row['sic_CDR'] < 0.001:
            temp[index] = 0
            
    elif row["h_piomas"] > 0:
        if row["h_cs2smos"] > 0:
            temp[index] = (row["h_cs2smos"] + row["h_piomas"])/2
        else: 
            temp[index] = row["h_piomas"]
            
    else:
         if row["h_cs2smos"] > 0:
            temp[index] = row["h_cs2smos"]
   
        

data["ice_thickness"] = temp            

# drop original thickness columns
data = data.drop(['h_piomas', 'h_cs2smos'], axis = 1)

# frop nan rows
data = data.dropna()

# convert x_EASE and y_EASE to lat and lon
# convert velocity components to magnitude and angle
lats = []
lons = []
buoy_vel_mags = []
buoy_vel_dirs = []
wind_vel_mags = []
wind_vel_dirs = []
for index, row in data.iterrows(): 
    # coordinate conversion
    x = row['x_EASE']
    y = row['y_EASE']
    lat, lon = interpolate_coordinate(x, y, grid)
    lats.append(lat)
    lons.append(lon)

    # buoy velocity conversion
    buoy_mag, buoy_dir = caonvert_vel_vector(row['u_buoy'], row['v_buoy'])
    buoy_vel_mags.append(buoy_mag)
    buoy_vel_dirs.append(buoy_dir)

    # wind velocity conversion
    wind_mag, wind_dir = caonvert_vel_vector(row['u_ERA5'], row['v_ERA5'])
    wind_vel_mags.append(wind_mag)
    wind_vel_dirs.append(wind_dir)

# add the converted data to dataset
data['buoy_lat'] = lats
data['buoy_lon'] = lons
data['buoy_vel_mag'] = buoy_vel_mags
data['buoy_vel_dir'] = buoy_vel_dirs
data['wind_vel_mag'] = wind_vel_mags
data['wind_vel_dir'] = wind_vel_dirs

# remove unwanted columns
# data = data.drop(['u_buoy', 'v_buoy', 'u_ERA5', 'v_ERA5', 'x_EASE', 'y_EASE'], axis = 1)  

# save the converted x_EASE and y_EASE to csv file
data.to_csv('data/converted.csv', index=False)  

In [ ]:
# read the dataset from the saved csv file to prevent running the above cell
data = pd.read_csv('data/converted.csv')  

In [ ]:
# normalize the thickness
data['ice_thick_norm'] =(data['ice_thickness'] - data['ice_thickness'].min()) / (data['ice_thickness'].max() - data['ice_thickness'].min())
data['buoy_vel_norm'] =(data['buoy_vel_mag'] - data['buoy_vel_mag'].min()) / (data['buoy_vel_mag'].max() - data['buoy_vel_mag'].min())

# Visualize the dataset as a time series

In [ ]:
years_list = list(pd.unique(data['year']))

if not os.path.exists('pictures/vis'):
    os.makedirs('pictures/vis')

for year in years_list:
        
    df = data[data['year'] == year]
    buoys_list = pd.unique(df['id_buoy'])

    fig = plt.figure(figsize=(8, 8), edgecolor='w')
    for b in buoys_list:
        df_b = df[df['id_buoy'] == b]
        lat, lon = df_b['buoy_lat'].to_list(), df_b['buoy_lon'].to_list()
        thick = df_b['ice_thick_norm'].to_list()

        visualize(lat, lon, grid, fig, projection='stere', 
                 show_grid=False, grid_res=4, width=5000000, height=5000000, 
                  show_text=True, year=year, thick=thick)

# monthly avergae for all buoys through years

In [ ]:
import random

if not os.path.exists('pictures'):
    os.makedirs('pictures')

if not os.path.exists('pictures/all'):
    os.makedirs('pictures/all')
    
buoys_list = list(pd.unique(data['id_buoy']))
years_list = list(pd.unique(data['year']))

# assign color to each buoy randomely
colors = []
r = random.random
for i in range(0,len(buoys_list)):
    rgb =  (r(),r(), 0)
    colors.append(rgb)

    
for year in years_list:
    # create the folders
    if not os.path.exists('pictures/{}'.format(year)):
        os.makedirs('pictures/{}'.format(year))

    df = data[data['year'] == year]
    month_list = list(pd.unique(df['month']))

    lat_avg = df.groupby(['month','id_buoy']).agg({'buoy_lat': ['mean']})     #agg({'buoy_lat': ['mean', 'min', 'max','std']})
    lon_avg = df.groupby(['month','id_buoy']).agg({'buoy_lon': ['mean']}) 
    thick_avg = df.groupby(['month','id_buoy']).agg({'ice_thick_norm': ['mean']}) 
    vel_avg = df.groupby(['month','id_buoy']).agg({'buoy_vel_norm': ['mean']}) 

    fig = plt.figure(figsize=(8, 8), edgecolor='w')
    for m in month_list:
        lat = lat_avg.loc[m, ('buoy_lat', 'mean')].values
        lon = lon_avg.loc[m, ('buoy_lon', 'mean')].values
        thick = thick_avg.loc[m, ('ice_thick_norm', 'mean')].values
        vel = vel_avg.loc[m, ('buoy_vel_norm', 'mean')].values
        current_buoys = list((lat_avg.loc[m, ('buoy_lat', 'mean')]).index)
        color = []
        for b in current_buoys:
            idx = buoys_list.index(b)
            color.append(colors[idx])

        draw_map(lat, lon, thick, color, grid, fig, projection='stere', 
                 show_grid=False, grid_res=4, width=4000000, height=4000000,
                 year=year, month=m, doy=[])


# motion of one buoy through years

In [ ]:
import random

if not os.path.exists('pictures'):
    os.makedirs('pictures')

if not os.path.exists('pictures/all'):
    os.makedirs('pictures/all')

    # find the most repeated buoys and plot their motion
dfl = data.groupby('id_buoy')['year'].nunique()
dfl = dfl.sort_values(ascending=False)
print(dfl)

buoy = 2416
df = data[data['id_buoy'] == buoy]
years_list = pd.unique(df['year'])

    
for year in years_list:
    if year < 2002:
        continue
    # create the folders
    if not os.path.exists('pictures/{}'.format(year)):
        os.makedirs('pictures/{}'.format(year))

    dfn = df[df['year'] == year]
    month_list = list(pd.unique(dfn['month']))

    lat = list(dfn['buoy_lat'].values)
    lon = list(dfn['buoy_lon'].values)
    thickness = list(dfn['ice_thick_norm'].values)
    doy = list(dfn['doy'].values)

    for i in range(len(lat)):
        fig = plt.figure(figsize=(8, 8), edgecolor='w')
        draw_map([lat[i]], [lon[i]], thickness[i] , thickness[i], grid, fig, projection='stere', 
                 show_grid=False, grid_res=4, width=4000000, height=4000000,
                 year=year, month=1, doy=doy[i])        
    

# read the provided test data and clean it up

In [7]:
data = read_dataset('data/DRIFT_DATA_TEST.csv')
#print(data.describe())
'''
data cleaning up
'''
# data['h_cs2smos'].isna().sum()
data = data.drop(['u_buoy', 'v_buoy', 'id_buoy'], axis=1)
# average the ice thickness from two measurements
temp = [None]*data.shape[0]
# iterate through all the rows and check if thickness is 0
for index, row in data.iterrows():
    if row["h_piomas"] == 0 :  
        if row["h_cs2smos"] > 0: 
            temp[index] = row["h_cs2smos"]
        else:
            temp[index] = 0
            
    elif row["h_piomas"] > 0:
        if row["h_cs2smos"] > 0:
            temp[index] = (row["h_cs2smos"] + row["h_piomas"])/2
        else: 
            temp[index] = row["h_piomas"]
            
    else:
         if row["h_cs2smos"] > 0:
            temp[index] = row["h_cs2smos"]
   
data["ice_thickness"] = temp            

# drop original thickness columns
data = data.drop(['h_piomas', 'h_cs2smos'], axis = 1)

# frop nan rows
data = data.dropna()

# convert x_EASE and y_EASE to lat and lon
# convert velocity components to magnitude and angle
lats = []
lons = []
buoy_vel_mags = []
buoy_vel_dirs = []
wind_vel_mags = []
wind_vel_dirs = []
for index, row in data.iterrows(): 
    # coordinate conversion
    x = row['x_EASE']
    y = row['y_EASE']
    lat, lon = interpolate_coordinate(x, y, grid)
    lats.append(lat)
    lons.append(lon)

    # wind velocity conversion
    wind_mag, wind_dir = caonvert_vel_vector(row['u_ERA5'], row['v_ERA5'])
    wind_vel_mags.append(wind_mag)
    wind_vel_dirs.append(wind_dir)

# add the converted data to dataset
data['buoy_lat'] = lats
data['buoy_lon'] = lons
data['wind_vel_mag'] = wind_vel_mags
data['wind_vel_dir'] = wind_vel_dirs

display(data)

# save the converted x_EASE and y_EASE to csv file
data.to_csv('data/converted_test.csv', index=False)  

,year,month,day,doy,x_EASE,y_EASE,u_ERA5,v_ERA5,sic_CDR,d2c,ice_thickness,buoy_lat,buoy_lon,wind_vel_mag,wind_vel_dir
0,1979,2,18,49,197.656311,204.507797,5.998414,3.617303,0.987723,375.766965,2.535649,83.278894,55.245082,7.004702,0.542654
1,1979,2,19,50,197.769897,204.840912,-1.414826,-0.201038,0.964051,370.636136,2.539519,83.263783,55.610016,1.429038,3.282756
2,1979,2,19,50,147.548553,157.382889,-4.140861,3.038851,1.000000,381.590523,3.746467,81.023992,-145.578490,5.136277,2.508495
3,1979,2,20,51,146.934814,120.546783,2.998362,4.055094,1.000000,413.672796,2.496566,74.509120,-119.765602,5.043210,0.934110
4,1979,2,21,52,197.534439,204.845886,-8.538108,4.243983,0.978987,376.255493,2.530706,83.295108,55.615465,9.534709,2.680297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84865,2019,12,30,364,51.783913,255.659637,-3.079250,8.509069,0.003184,25.612993,0.019423,56.018602,149.608252,9.049090,1.918014
84866,2019,12,30,364,195.420746,101.970360,-7.462417,-2.639882,0.995414,177.841985,1.086896,71.777914,-79.118458,7.915595,3.481625
84867,2019,12,30,364,147.647980,104.794327,-1.493360,-3.978977,0.986766,21.720861,1.228123,71.296458,-113.692218,4.249986,4.353359
84868,2019,12,30,364,200.005966,174.816803,-1.934895,-1.281953,1.000000,527.681133,1.430554,85.290268,-14.524094,2.321039,3.726751
